In [1]:
#The purpose of this notebook is to perform exploratory data analysis and exercise data cleaning/preprocessing on a dataset.
#The absenteeism dataset used in this project tracks the number of hours that an employee is absent from work, and records
#a numerical reason for absence from 0 through 28. The ultimate goal of this project is to create a regression model in Python
#to find which input variables within the dataset contribute to an increased probability of an employee missing time from work.\

#Portions of this jupyter notebook that output large dataframes were commented out in order to preserve readability.

In [2]:
import pandas as pd

In [3]:
raw_csv_data = pd.read_csv('Absenteeism_data.csv')

In [4]:
#Previewing the dataset
raw_csv_data.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2


In [5]:
#Assigning the raw dataset to a new dataframe, df
df = raw_csv_data.copy()

In [6]:
#Testing that the copy was successful
#df

In [7]:
#Displaying the dataframe in its entirety
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
#display(df)

In [9]:
#Viewing data summary to see if any missing values exist in the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
ID                           700 non-null int64
Reason for Absence           700 non-null int64
Date                         700 non-null object
Transportation Expense       700 non-null int64
Distance to Work             700 non-null int64
Age                          700 non-null int64
Daily Work Load Average      700 non-null float64
Body Mass Index              700 non-null int64
Education                    700 non-null int64
Children                     700 non-null int64
Pets                         700 non-null int64
Absenteeism Time in Hours    700 non-null int64
dtypes: float64(1), int64(10), object(1)
memory usage: 65.7+ KB


# Removing Irrelevant Data

In [10]:
#Dropping the variable "ID" from the dataframe. This unique identifier is only a label variable and will not provide meaningful
#value to the analysis.

df = df.drop(["ID"], axis = 1)
#df

In [11]:
#Isolating 'Reason for Absence' variable for analysis
#df['Reason for Absence']

In [12]:
#Finding the minimum and maximum values within the variable
df['Reason for Absence'].min()

0

In [13]:
df['Reason for Absence'].max()

28

In [14]:
#Extract the distinct values from the 'Reason for Absence' variable
df["Reason for Absence"].unique()

array([26,  0, 23,  7, 22, 19,  1, 11, 14, 21, 10, 13, 28, 18, 25, 24,  6,
       27, 17,  8, 12,  5,  9, 15,  4,  3,  2, 16], dtype=int64)

In [15]:
#The len() function returns the number of elements in the variable
len(df["Reason for Absence"].unique())

28

In [16]:
#Sorting the unique values to find any values that are missing. 
#Doing this reveals there is no value of 20 in the "Reason for Absence" variable.
sorted(df["Reason for Absence"].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28]

# Creating Dummy Variables

In [17]:
#Save dummy variables into new dataframe
reason_columns = pd.get_dummies(df["Reason for Absence"])

In [18]:
#reason_columns

In [19]:
#Create a new aggregate column to check for missing/errant values
#The new column should have a sum of 1.
reason_columns["Check"] = reason_columns.sum(axis = 1)
#reason_columns

In [20]:
#Using the sum method for the "Check" column to verify no missing values
#The sum of the vertical axis should be 700.
reason_columns["Check"].sum(axis = 0)

700

In [21]:
#The unique() function can also verify that 1 is the only unique value in the column.
#Output of 1 confirms this.
reason_columns["Check"].unique()

array([1], dtype=int64)

In [22]:
#Delete the Check column since it is no longer needed.
reason_columns = reason_columns.drop(["Check"], axis = 1)
#reason_columns

In [23]:
#Dropping Reason = 0 to avoid multicollinearity issues
reason_columns = pd.get_dummies(df["Reason for Absence"], drop_first = True)
#reason_columns

# Grouping Reasons for Absence into Categories

In [24]:
df.columns.values

array(['Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets',
       'Absenteeism Time in Hours'], dtype=object)

In [25]:
reason_columns.columns.values

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 21, 22, 23, 24, 25, 26, 27, 28], dtype=int64)

In [26]:
#Drop the original "Reason for Absence" column
df = df.drop(["Reason for Absence"], axis = 1)
#df

In [27]:
#reason_columns.loc[:, 1:14].max(axis = 1)

In [28]:
#Grouped the dummy reasons into 4 categories based on description.
#Use the .loc() and .max() functions to return 1 if the value of 1 was present in
#reasons 1-14, 15-17, 18-21, and 22-28.
reason_type_1 = reason_columns.loc[:, 1:14].max(axis = 1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis = 1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis = 1)
reason_type_4 = reason_columns.loc[:, 22:28].max(axis = 1)

# Combining the reason_type columns to the original dataframe

In [29]:
#use the pd.concat() function to concatenate dataframes
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)
#df

In [30]:
df.columns.values

array(['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 0, 1, 2, 3],
      dtype=object)

In [31]:
#Assign new column names to the concatenated dataframe
column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', "Reason_1", "Reason_2", "Reason_3", "Reason_4"]

df.columns = column_names

In [32]:
df.head()

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason_1,Reason_2,Reason_3,Reason_4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1
3,16/07/2015,279,5,39,239.554,24,1,2,0,4,1,0,0,0
4,23/07/2015,289,36,33,239.554,30,1,2,1,2,0,0,0,1


In [33]:
#Reorder the column names
column_names_reordered = ["Reason_1", "Reason_2", "Reason_3", "Reason_4", 'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

df  = df[column_names_reordered]

In [34]:
df.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,23/07/2015,289,36,33,239.554,30,1,2,1,2


# Creating a checkpoint

In [35]:
#Create a copy of the current state of the dataframe as a checkpoint.
df_reason_mod = df.copy()
#df_reason_mod

# Manipulating the Date variable

In [36]:
#Verifying the data type of the Date column
type(df_reason_mod["Date"])

pandas.core.series.Series

In [37]:
type(df_reason_mod["Date"][0])

str

In [38]:
#Converting string date values to timestamp
df_reason_mod["Date"] = pd.to_datetime(df_reason_mod["Date"], format = "%d/%m/%Y")
#df_reason_mod["Date"]

In [39]:
df_reason_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 14 columns):
Reason_1                     700 non-null uint8
Reason_2                     700 non-null uint8
Reason_3                     700 non-null uint8
Reason_4                     700 non-null uint8
Date                         700 non-null datetime64[ns]
Transportation Expense       700 non-null int64
Distance to Work             700 non-null int64
Age                          700 non-null int64
Daily Work Load Average      700 non-null float64
Body Mass Index              700 non-null int64
Education                    700 non-null int64
Children                     700 non-null int64
Pets                         700 non-null int64
Absenteeism Time in Hours    700 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(8), uint8(4)
memory usage: 57.5 KB


# Creating the Month column

In [40]:
df_reason_mod["Date"][0].month

7

In [41]:
#Create an empty list to store month values
list_months = []
list_months

[]

In [42]:
#Iterate through the number of rows in the dataframe with .shape(),
#then append the output to the empty list_months list
for i in range(df_reason_mod.shape[0]):
    list_months.append(df_reason_mod["Date"][i].month)
    
#list_months

In [43]:
len(list_months)

700

In [44]:
#Add the list_months list as a new column to the original dataframe named "Month Value"
df_reason_mod["Month Value"] = list_months

In [45]:
#Check to verify new column was added
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7


# Extracting the day of the week

In [46]:
#Using the .weekday() function to output the day of the week from the date column
df_reason_mod["Date"][0].weekday()

1

In [47]:
#Create a function that returns the weekday
def date_to_weekday(date_value):
    return date_value.weekday()

In [48]:
#Create the "Day of the Week" column and apply the date_to_weekday function to it
df_reason_mod["Day of the Week"] = df_reason_mod["Date"].apply(date_to_weekday)

In [49]:
#Verify the function operated correctly
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month Value,Day of the Week
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7,3


In [50]:
#Dropping the Date column from the dataframe
df_reason_mod = df_reason_mod.drop(["Date"], axis = 1)
#df_reason_mod

In [51]:
#Reorder the columns to move "Month Value" and "Day of the Week" to replace the Date column that was dropped
column_reorder = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Day of the Week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [52]:
df_reason_mod = df_reason_mod[column_reorder]
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,1,2,1,2


# Checkpoint 2

In [53]:
df_reason_date_mod = df_reason_mod.copy()

In [54]:
#Using the .map() function to transform the "Education" column into 2 different values.
#0 = High School Graduate, 1 = Bachelor's or higher
df_reason_date_mod["Education"] = df_reason_date_mod["Education"].map({1:0, 2:1, 3:1, 4:1})

In [55]:
#Verifying only 0 and 1 as values
df_reason_date_mod["Education"].value_counts()

0    583
1    117
Name: Education, dtype: int64

# Final Checkpoint

In [56]:
df_preprocessed = df_reason_date_mod.copy()

In [57]:
df_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [58]:
#Export the preprocessed dataframe as a .csv 
df_preprocessed.to_csv("Absenteeism_Preprocessed.csv", index = False)